# Collision Avoidance - Live Demo

Setup
---

In [1]:
###-----
import torch
import torchvision

model = torchvision.models.alexnet(pretrained=False)
model.classifier[6] = torch.nn.Linear(model.classifier[6].in_features, 2)
model.load_state_dict(torch.load('best_model.pth'))
device = torch.device('cuda')
model = model.to(device)

###------
import cv2
import numpy as np

mean = 255.0 * np.array([0.485, 0.456, 0.406])
stdev = 255.0 * np.array([0.229, 0.224, 0.225])

normalize = torchvision.transforms.Normalize(mean, stdev)

def preprocess(camera_value):
    global device, normalize
    x = camera_value
    x = cv2.cvtColor(x, cv2.COLOR_BGR2RGB)
    x = x.transpose((2, 0, 1))
    x = torch.from_numpy(x).float()
    x = normalize(x)
    x = x.to(device)
    x = x[None, ...]
    return x

print('ok')

ok


Camera
---

In [11]:
###-----
import traitlets
from IPython.display import display
import ipywidgets.widgets as widgets
from jetbot import Camera, bgr8_to_jpeg

camera = Camera.instance(width=224, height=224, fps=10)
image = widgets.Image(format='jpeg', width=224, height=224)
blocked_slider = widgets.FloatSlider(description='blocked', min=0.0, max=1.0, orientation='vertical')
#display(widgets.HBox([image, blocked_slider]))

try:
    while camera_links: camera_links.pop().unlink()
except: pass
camera_links = [traitlets.dlink((camera, 'value'), (image, 'value'), transform=bgr8_to_jpeg)]
print("camera_links",len(camera_links))
print("ok")

camera_links 1
ok


Run
---

In [12]:
###-----
from jetbot import Robot
robot = Robot()

###-----
import torch.nn.functional as F
import time

def update(change):
    global blocked_slider, robot
    x = change['new'] 
    x = preprocess(x)
    y = model(x)
    
    # we apply the `softmax` function to normalize the output vector so it sums to 1 (which makes it a probability distribution)
    y = F.softmax(y, dim=1)
    
    prob_blocked = float(y.flatten()[0])
    
    blocked_slider.value = prob_blocked
    
    if prob_blocked < 0.1:
        #robot.forward(0.3)
        robot.left_motor.value = 0.33
        robot.right_motor.value = 0.3
    else:
        robot.left(0.3)
    
    time.sleep(0.001)

    
update({'new': camera.value})  # we call the function once to intialize
camera.observe(update, names='value')  # this attaches the 'update' function to the 'value' traitlet of our camera

print('ok')
display(widgets.HBox([image, blocked_slider]))

ok


Awesome! If your robot is plugged in it should now be generating new commands with each new camera frame.  Perhaps start by placing your robot on the ground and seeing what it does when it reaches an obstacle.

If you want to stop this behavior, you can unattach this callback by executing the code below.

Stop
---

In [10]:
import time

try: camera.unobserve_all()
except: pass

time.sleep(0.1)  # add a small sleep to make sure frames have finished processing
robot.stop()